In [ ]:
import itertools
import sys
import os

sys.path.append("../..")

import numpy as np
import pandas as pd
import scanpy as sc
from src.data.preprocess_data import preprocess
from matplotlib import pyplot as plt
from scanpy._utils import _check_use_raw
from src.utils.utils import get_gene_list_real_data, get_test_statistics
from src.scoring_methods.gene_signature_scoring import score_signature
from data.constants import BASE_PATH_DATA, BASE_PATH_EXPERIMENTS

sc.settings.verbosity = 1

In [ ]:
# adata_healthy = sc.read_h5ad('../data/real_data/healthy.h5ad')
# adata_unhealthy = sc.read_h5ad('../data/real_data/unhealthy.h5ad')
# adata = adata_healthy.concatenate(adata_unhealthy)


In [ ]:
# dict_samples = {}


In [ ]:
# for group in adata.obs.groupby(by='sample_id'):
#     dict_samples[group[0]] = adata[group[1].index, :].copy()


In [ ]:
# del adata
# del adata_unhealthy
# del adata_healthy


In [ ]:
# dict_samples.keys()


In [ ]:
# adata = dict_samples['P23T']
adata = sc.read_h5ad('../data/real_data/P23T_adata.h5ad')


In [ ]:
adata = adata[adata.obs['healthy'] != 'undecided', :]


In [ ]:
preprocess(adata, min_genes=500, min_cells=10,
           target_sum=1e4, copy=False, verbose=1, log=None)


In [ ]:
gene_list = get_gene_list_real_data(
    adata,
    dge_method="wilcoxon",
    dge_key="wilcoxon",
    dge_pval_cutoff=0.01,
    dge_log2fc_min=0.0025,
    nr_de_genes=100,
    mode="random",
    log='get_gene_list',
    copy=False,
    verbose=1
)


In [ ]:
# precalc_ranks = None
maxRank = 1500
# w_neg = 1
# name = "_UCell"
# assay = "counts"
# chunk_size = 1000
# BPPARAM = None
# ncores = 1
ties_method = "average"
# force_gc = False
copy = False
use_raw = False
random_state = 5


In [ ]:
adata = adata.copy() if copy else adata
use_raw = _check_use_raw(adata, use_raw)

if random_state is not None:
    np.random.seed(random_state)

# remove genes from gene_list not available in the data
var_names = adata.raw.var_names if use_raw else adata.var_names
var_names = var_names.tolist()
gene_list = list((set(var_names)).intersection(set(gene_list)))
genes_to_ignore = list((set(gene_list)).difference(set(var_names)))
if len(genes_to_ignore) > 0:
    sc.logging.warning(f"genes are not in var_names and ignored: {genes_to_ignore}")
if len(gene_list) == 0:
    raise ValueError("No valid genes were passed for scoring.")

In [ ]:
len(adata.var_names)

In [ ]:
if not isinstance(maxRank, int):
    raise ValueError(f'maxRank {maxRank} must be of type int')

In [ ]:
if maxRank > len(adata.var_names):
    print(f'Provided maxRank is larger than the number of available genes. Set maxRank=len(adata.var_names)')
    maxRank=len(adata.var_names)

In [ ]:
if len(gene_list)> maxRank:
    raise ValueError(f'The provided signature contains more genes than the maxRank parameter. Increase the maxRank parameter or choose a shorter signature.')

Apply the scoring functionality on adata.X . For know we avaid the following usage:
- reuse of precomputed ranks
- parallelization of score computation for different chunks of the data 
- use of forced garbage collector, i.e., the direct deletion of unused variable for memory 
- negative weighted signatures

In [ ]:
data_df = pd.DataFrame(adata.X.todense(), index=adata.obs_names, columns=adata.var_names)

In [ ]:
data_df_small = data_df.iloc[0:10, 0:10]

In [ ]:
data_df_small

In [ ]:
# get ranks data table
ranked_data_df = data_df_small.apply(func=(lambda x: x.rank(ascending=False, na_option='bottom')), axis=1)


In [ ]:
ranked_data_df

In [ ]:
ranked_data_df = data_df.apply(func=(lambda x: x.rank(ascending=False, na_option='bottom')), axis=1)

In [ ]:
for i in np.random.randint(low=0, high=ranked_data_df.shape[0], size=5):
    # print(ranked_data_df.iloc[i,:].describe())
    ranked_data_df.iloc[i,:].hist(bins = 50)
    plt.show()

In [ ]:
min_rank = np.inf
max_rank = -np.inf
index_max_rank = None
for index, row in ranked_data_df.iterrows():
    row_max_rank = row[row<7000].max()
    row_min_rank = row[row<7000].min()
    if row_min_rank < min_rank:
        min_rank = row_min_rank
    if row_max_rank > max_rank:
        max_rank = row_max_rank
        index_max_rank = index

print('index maxRank: ', index_max_rank)
print(min_rank, max_rank)

In [ ]:
ranked_data_df.loc['P23T-E-AGCTCTCTCTCAAGTG-1',:].hist()

In [ ]:
# get u_stat for signature 
ranked_data_df_signature = ranked_data_df.loc[:,gene_list]

In [ ]:
ranked_data_df_signature

In [ ]:
def u_stat(rank_value, maxRank=1500):
    insig = rank_value > maxRank
    if all(insig):
        return 0 
    else:
        rank_value[insig] = maxRank+1
        rank_sum = rank_value.sum()
        len_sig  = len(rank_value)
        u_value = rank_sum - (len_sig * (len_sig + 1))/2
        auc = 1 - u_value/(len_sig * maxRank)
        return auc

In [ ]:
scores = ranked_data_df_signature.apply(func=(lambda x: u_stat(x, maxRank=1500)),axis=1)

In [ ]:
adata.obs['UCell scoring'] = scores

In [ ]:
# scores[adata.obs['healthy']=='healthy'].hist(density=True, label = 'healthy', alpha=0.75)
# scores[adata.obs['healthy']=='unhealthy'].hist(density=True, label='unhealthy', alpha=0.75)
# plt.title("UCell scores for random upregulated signature genes.")
# plt.legend()


In [ ]:
from src.scoring_methods.tirosh_signature_scoring import score_genes as tirosh_scoring
from src.scoring_methods.ucell_signature_scoring import score_genes as ucell_scoring

In [ ]:
tirosh_scoring(
        adata,
        gene_list,
        n_bins=25,
        ctrl_size=100,
        verbose=0,
        score_name="Tirosh scoring",
        random_state=5,
    )

In [ ]:
ucell_scoring(
        adata,
        gene_list,
        maxRank= 1500,
        ties_method= "average",
        verbose=0,
        score_name="UCell scoring 2",
        random_state=5,
    )

In [ ]:
adata.obs["UCell scoring"][adata.obs["healthy"]=='healthy'].hist(bins=50, density=True, label="healthy cells", alpha=0.5)
adata.obs["UCell scoring"][adata.obs["healthy"]=='unhealthy'].hist(bins=50,density=True, label="unhealthy cells", alpha=0.5)
plt.legend()

In [ ]:
adata.obs["UCell scoring 2"][adata.obs["healthy"]=='healthy'].hist(bins=50, density=True, label="healthy cells", alpha=0.5)
adata.obs["UCell scoring 2"][adata.obs["healthy"]=='unhealthy'].hist(bins=50,density=True, label="unhealthy cells", alpha=0.5)
plt.legend()

In [ ]:
adata.obs["Tirosh scoring"][adata.obs["healthy"]=='healthy'].hist(bins=50, density=True, label="healthy cells", alpha=0.5)
adata.obs["Tirosh scoring"][adata.obs["healthy"]=='unhealthy'].hist(bins=50,density=True, label="unhealthy cells", alpha=0.5)
plt.legend()

In [ ]:
test_stat = get_test_statistics(adata,
                                ['Tirosh scoring', 'UCell scoring', 'UCell scoring 2'],
                                test_method="auc",
                                label_col='healthy',
                                label_whsc='unhealthy',
                                save=False,
                                store_data_path=None)

In [ ]:
test_stat

The following section compares differemnt maxRank values

In [ ]:
from src.scoring_methods.ucell_signature_scoring import score_genes as ucell_scoring

In [ ]:
maxRanks = [1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500]

In [ ]:
for maxRank in maxRanks:
    ucell_scoring(
            adata,
            gene_list,
            maxRank= maxRank,
            ties_method= "average",
            verbose=0,
            score_name="UCell_"+str(maxRank),
            random_state=5,
        )

In [ ]:
sc_name = ["UCell_"+str(maxRank) for maxRank in maxRanks]

In [ ]:
test_stat = get_test_statistics(adata,
                                sc_name,
                                test_method="auc",
                                label_col='healthy',
                                label_whsc='unhealthy',
                                save=False,
                                store_data_path=None)

In [ ]:
test_stat.sort_values(by='Statistic', ascending=False)

In [ ]:
test_stat.loc[test_stat['Scoring method']==sc_n.lower(),:]['Statistic']

In [ ]:
for sc_n in sc_name:
    plt.hist(adata.obs[sc_n][adata.obs['healthy']=='healthy'], alpha=0.5, density=True,label='healthy')
    plt.hist(adata.obs[sc_n][adata.obs['healthy']=='unhealthy'], alpha=0.5, density=True,label='unhealthy')
    plt.legend()
    plt.title(f"{sc_n} achieved AUC of {test_stat.loc[test_stat['Scoring method']==sc_n.lower(),:]['Statistic']}")
    plt.show()

### Construct UCell scoring for sparse matrix

In [ ]:
adata = sc.read_h5ad(os.path.join(BASE_PATH_DATA, 'real_data/multi.h5ad'))
adata = adata.raw.to_adata()
adata.var_names = adata.var['_index']
adata.var_names.name = None

In [ ]:
adatas = {}
for group in adata.obs.groupby('orig.ident'):
    adatas[group[0]] = adata[group[1].index,].copy()
del adata

for key, adata in adatas.items():
    preprocess(adata,
               min_genes=500,
               min_cells=5,
               target_sum=1e4)

In [ ]:
adata = sc.concat(list(adatas.values()), join='inner')

In [ ]:
adata

In [ ]:
curr_adata = adatas['P1_0']
del adatas

In [ ]:
DE_of_celltypes = pd.read_csv(os.path.join(BASE_PATH_DATA, 'real_data/DE_by_celltype.csv'))

In [ ]:
gene_list = DE_of_celltypes[DE_of_celltypes['Cell Type']=='CD4 Proliferating'].nlargest(20, columns=['Average Log Fold Change'])['Gene'].values.tolist()

In [ ]:
gene_list

In [ ]:
from scipy.sparse import issparse
from tqdm import tqdm
from scipy.stats import rankdata
from datetime import datetime



Compare method on 1 sample

In [ ]:
curr_adata.X

In [ ]:
data_df = pd.DataFrame.sparse.from_spmatrix(curr_adata.X, index=curr_adata.obs_names, columns=curr_adata.var_names)

In [ ]:
gene_list_idx = np.where(data_df.columns.isin(gene_list))[0]
gene_list_idx

In [ ]:
%%time 

res = data_df.apply(
    lambda x: x.sparse.to_dense().rank(method='average', ascending=False, na_option="bottom"),
    axis=1,
    raw=False
)

# does not terminate in 2 min, one sample

In [ ]:
%%time 

res = data_df.apply(
    lambda x: rankdata(-x),
    axis=1,
    raw=True,
    result_type='expand'
)
# using raw one sample, one sample

In [ ]:
from datetime import datetime

for bs in [1000, 1500, 2000, 2500,3000,3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000]:
    start = datetime.now()
    groupsi = pd.cut(np.arange(curr_adata.obs.shape[0]),(curr_adata.obs.shape[0]//bs +1), labels=False)
    rankes = []
    for group in tqdm(curr_adata.obs.groupby(groupsi)):
        data_df = pd.DataFrame.sparse.from_spmatrix(curr_adata[group[1].index,].X, index=group[1].index, columns=curr_adata.var_names)
        res = data_df.apply(
            lambda x: rankdata(-x),
            axis=1,
            raw=True
        )
        rankes.append(res[gene_list])
    end = datetime.now()
    print(f'for bs={bs} needed {end-start} time')

    
# using one sample with raw but separate sample into managable batches --> find best batch size

In [ ]:
for bs in [1000, 1500, 2000, 2500,3000,3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000]:
    start = datetime.now()
    groupsi = pd.cut(np.arange(curr_adata.obs.shape[0]),(curr_adata.obs.shape[0]//bs +1), labels=False)
    rankes = []
    for group in tqdm(curr_adata.obs.groupby(groupsi)):
        data_df = pd.DataFrame(
            adata[group[1].index,].X.todense(), index=group[1].index, columns=adata.var_names
        )
        res = data_df.apply(
            lambda x: rankdata(-x),
            axis=1,
            raw=True
        )
        rankes.append(res[gene_list])
    end = datetime.now()
    print(f'for bs={bs} needed {end-start} time')

    
# using one sample with raw but separate sample into managable batches --> find best batch size

Switch from one sample to all the batches together

In [ ]:
for bs in [6000, 6500, 7000]:
    start = datetime.now()
    
    groupsi = pd.cut(np.arange(adata.obs.shape[0]),(adata.obs.shape[0]//bs +1), labels=False)

    rankes = []
    for group in tqdm(adata.obs.groupby(groupsi)):
        data_df = pd.DataFrame.sparse.from_spmatrix(adata[group[1].index,].X, index=group[1].index, columns=adata.var_names)
        res = data_df.apply(
            lambda x: rankdata(-x),
            axis=1,
            raw=True
        )
        rankes.append(res[gene_list])
    
    end = datetime.now()
    print(f'for bs={bs} needed {end-start} time')

## creating DataFrame.sparse for each partition

In [ ]:
#for bs in [4000, 4500, 5000, 5500, 6000, 6500, 7000]:
#for bs in [500, 750, 1000,1250,1500,5500]:
for bs in [1000]:
    start = datetime.now()
    
    groupsi = pd.cut(np.arange(adata.obs.shape[0]),(adata.obs.shape[0]//bs +1), labels=False)

    rankes = []
    for group in tqdm(adata.obs.groupby(groupsi)):
        
        data_df = pd.DataFrame(
            adata[group[1].index,].X.todense(), index=group[1].index, columns=adata.var_names
        )
        res = data_df.apply(
            lambda x: rankdata(-x),
            axis=1,
            raw=True
        )
        rankes.append(res[gene_list])
    
    end = datetime.now()
    print(f'for bs={bs} needed {end-start} time')

## creating DataFrame for each densified partition

Compute ustat and scores on the ranks

In [ ]:
def u_stat(rank_value, maxRank=1500):
    insig = rank_value > maxRank
    if all(insig):
        return 0
    else:
        rank_value[insig] = maxRank + 1
        rank_sum = rank_value.sum()
        len_sig = len(rank_value)
        u_value = rank_sum - (len_sig * (len_sig + 1)) / 2
        auc = 1 - u_value / (len_sig * maxRank)
        return auc

In [ ]:
# get ranks for signautre genes
ranked_data_df_signature = pd.concat(rankes, axis=0)

In [ ]:
# compute u_stat for each cell over all genes
score = ranked_data_df_signature.apply(
    func=(lambda x: u_stat(x, maxRank=1500)), 
    axis=1,
    raw=True
)

In [ ]:
score

In [ ]:
score[adata.obs['celltype.l3']=='CD4 Proliferating'].hist(density=True, alpha=0.5, label='CD4 Proliferating')
score[adata.obs['celltype.l3']!='CD4 Proliferating'].hist(density=True, alpha=0.5, label=' not CD4 Proliferating')

In [ ]:
adata.obs['ucell_manual'] = score

In [ ]:
%%time
score_signature('ucell_scoring', adata, gene_list, score_name='ucell_new')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### Make Ucell faster

In [ ]:
# orig_adata = sc.read_h5ad(os.path.join(BASE_PATH_DATA, 'real_data/esophag/data.h5ad'))
# orig_adata = orig_adata[orig_adata.obs.malignant_key != 'undecided', :]
# orig_adata

In [ ]:
# adatas = {}
# for group in orig_adata.obs.groupby('sample_id'):
#     adatas[group[0]] = orig_adata[group[1].index,].copy()

# del orig_adata
# for key, adata in adatas.items():
#     preprocess(adata,
#                min_genes=200,
#                min_cells=1,
#                target_sum=1e4)
# orig_adata = sc.concat(list(adatas.values()), join='inner', merge='same')
# del adatas, adata
# orig_adata

In [ ]:
# sc.tl.pca(orig_adata)
# sc.pp.neighbors(orig_adata)
# sc.tl.umap(orig_adata)

In [ ]:
# orig_adata.write_h5ad(os.path.join(BASE_PATH_DATA, 'real_data/esophag/preoprocessed_data.h5ad'))

In [ ]:
orig_adata = sc.read_h5ad(os.path.join(BASE_PATH_DATA, 'real_data/esophag/preoprocessed_data.h5ad'))

In [ ]:
mes_sig = pd.read_csv(os.path.join(BASE_PATH_DATA, 'annotations/esophag/genesig_Mes.csv')).Mes.tolist()

In [ ]:
%%time
score_signature(method="ucell_scoring",
                adata=orig_adata,
                gene_list=mes_sig,
                score_name='mes_sig_scores'
                )

In [ ]:
import sys
from typing import Optional, Sequence

import numpy as np
import pandas as pd
import scanpy as sc
from anndata import AnnData
from scanpy._utils import AnyRandom, _check_use_raw
from scipy.sparse import issparse, csr_matrix
from scipy.stats import rankdata
import multiprocessing
from joblib import Parallel, delayed
import warnings
import os, psutil
from memory_profiler import memory_usage

sys.path.append("../..")

from src.utils.utils import check_signature_genes


def u_stat(rank_value, maxRank=1500):
    insig = rank_value > maxRank
    if all(insig):
        return 0
    else:
        rank_value[insig] = maxRank + 1
        rank_sum = rank_value.sum()
        len_sig = len(rank_value)
        u_value = rank_sum - (len_sig * (len_sig + 1)) / 2
        auc = 1 - u_value / (len_sig * maxRank)
        return auc

def compute_ranks_and_ustat(X_data, index, columns,gene_list,X_indices=None,X_indptr=None,X_shape=None,maxRank=1500):
    
    if any([x is None for x in [X_indices,X_indptr, X_shape]]):
        ptint('we should not enter here')
        data_df = pd.DataFrame(
            X_data, index=index, columns=columns
        )
    else:
        data_df = pd.DataFrame(
            csr_matrix((X_data, X_indices,X_indptr),X_shape, copy=True).todense(), index=index, columns=columns
        )

    res = (data_df.apply(
        lambda x: rankdata(-x),
        axis=1,
        raw=True
    ))[gene_list]
    
    del data_df
    
    score = res.apply(
        func=(lambda x: u_stat(x, maxRank=maxRank)),
        axis=1,
        raw=True
    )
    return score

def score_genes(
        adata: AnnData,
        gene_list: Sequence[str],
        maxRank: int = 1500,
        bs: int = 500,
        score_name: str = "score",
        random_state: AnyRandom = 0,
        copy: bool = False,
        use_raw: Optional[bool] = None,
        verbose: int = 0,
) -> Optional[AnnData]:
    start = sc.logging.info(f"computing score {score_name!r}")
    if verbose > 0:
        print(f"computing score {score_name!r}")

    adata = adata.copy() if copy else adata

    use_raw = _check_use_raw(adata, use_raw)

    _adata = adata.raw if use_raw else adata

    if random_state is not None:
        np.random.seed(random_state)

    # remove genes from gene_list not available in the data
    gene_list = check_signature_genes(_adata.var_names, gene_list)

    # check type of rank
    if not isinstance(maxRank, int):
        raise ValueError(f"maxRank {maxRank} must be of type int")

    # check maxRank is not larger than available nr. of genes
    if maxRank > len(_adata.var_names):
        print(
            f"Provided maxRank is larger than the number of available genes. Set maxRank=len(adata.var_names)"
        )
        maxRank = len(_adata.var_names)

    # check that signature is not longer than maxRank
    if len(gene_list) > maxRank and len(gene_list) <= len(_adata.var_names):
        warnings.warn(
            f"The provided signature contains more genes than the maxRank parameter. maxRank is increased to signature length"
        )
        maxRank = len(gene_list)
        

    sparse_X = issparse(_adata.X)

    # create groups of managable sizes
    bss = pd.cut(np.arange(_adata.obs.shape[0]), (_adata.obs.shape[0] // bs + 1), labels=False)
    
    num_cores = multiprocessing.cpu_count()
#     num_cores = 4
                           
    scores = Parallel(n_jobs=num_cores, require='sharedmem')(
        delayed(compute_ranks_and_ustat)(X_data=_adata[group[1].index,].X.data if sparse_X else _adata[group[1].index,].X,
                                         X_indices=_adata[group[1].index,].X.indices if sparse_X else None,
                                         X_indptr=_adata[group[1].index,].X.indptr if sparse_X else None,
                                         X_shape = _adata[group[1].index,].X.shape  if sparse_X else None,
                                         index = group[1].index,
                                         columns = _adata.var_names,
                                         gene_list = gene_list, 
                                         maxRank = maxRank) for group in _adata.obs.groupby(bss))

    scores = pd.concat(scores)
    
    adata.obs[score_name] = scores

    sc.logging.info(
        "    finished",
        time=start,
        deep=("added\n" f"    {score_name!r}, score of gene set (adata.obs)."),
    )
    return adata if copy else None

In [ ]:
%%time
scores=score_genes(orig_adata, mes_sig, score_name='mes_sig_scores_refactored')
# memory_usage((score_genes, [orig_adata, mes_sig],{'score_name':'mes_sig_scores_refactored'}))